In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import re
import pymysql
from sqlalchemy import create_engine
from functools import reduce
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import pymysql
from sqlalchemy import create_engine
# 导入模型
import xgboost as xgb
# 模型调参的工具
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import GridSearchCV
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
# Error metrics 评价指标
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pylab as plt

In [2]:
# 数据库信息
mysql_setting = {
    'host': '47.100.201.211',
    'port': 3306,
    'user': 'root',
    'passwd': 'iyGfLR64Ne4Ddhk7',
    # 数据库名称
    'db': 'data',
    'charset': 'utf8'
}
engine = create_engine("mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting), max_overflow=5)

In [3]:
# 滑动窗口特征
def mov_df(df, num):
    df['mov_sales_avg'] = df['dayPrice'].transform(lambda x: x.rolling(num).mean()) 
    df['mov_sales_max'] = df['dayPrice'].transform(lambda x: x.rolling(num).max())
    df['mov_sales_min'] = df['dayPrice'].transform(lambda x: x.rolling(num).min())
    return df
# 滑动窗口波动率特征
def mov_rate(df):
    df['滑窗价格波动'] = df['mov_sales_avg'].diff(periods=1)
    l1 = df['mov_sales_avg'].tolist()
    l2 = df['滑窗价格波动'].tolist()
    def rate(list1, list2):
        l = ['']
        i = 1
        while i <= len(list2)-1:
            r = list2[i]/list1[i-1]
            l.append(r)
            i = i + 1
        return l
    df['滑窗价格波动率'] = rate(l1, l2)
    df['滑窗价格波动率'] = pd.to_numeric(df['滑窗价格波动率'])
    return df
# 分割训练集预测集
def train(df):
    x_train = df.iloc[1:,2:]
    y_train = df.iloc[1:,1]
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    x_predict = df.iloc[-4:,2:]
    x_predict = x_predict.reset_index(drop=True)
    return [x_train, y_train, x_predict]

In [4]:
# 填充缺失价格，去重重复天数价格
def fillPrice(df):
    # datetime.strptime() 把datetime转化为str格式
    df['updateTime'] = df['updateTime'].apply(lambda x: x.strftime('%Y-%m-%d'))
    date_start = df['updateTime'].min()
    date_end = datetime.datetime.now().strftime('%Y-%m-%d')
    #根据最小日期和最大日期，计算日期间隔，由于date_start和date_end为string类型，因此需要先更改为日期类型
    delta = datetime.datetime.strptime(date_end, "%Y-%m-%d")-datetime.datetime.strptime(str(date_start), "%Y-%m-%d")
    for i in range(1,delta.days+1):
        date = (datetime.datetime.strptime(date_end, "%Y-%m-%d")-datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        #如果数据缺失，则补数
        if(date not in df['updateTime'].values):
            new_date = pd.DataFrame({'updateTime':date,'dayPrice':np.nan}, index=[0])
            df = df.append(new_date)
    df.sort_values(by="updateTime", inplace=True, ascending=True) 
    df['dayPrice'] = df['dayPrice'].fillna(df['dayPrice'].interpolate().round(2))
    df['updateTime'] = pd.to_datetime(df['updateTime'])
    df['updateTime'] = df['updateTime'].dt.strftime("%Y-%m")
    df = df.groupby('updateTime').mean('dayPrice').round(2)
    df.reset_index(level=0, inplace=True)
    return df

In [5]:
# 长期预测线上商品
def predict_online(url, product, w):
    if ('jd' in url):
        sql_cmd = "SELECT updateTime, dayPrice FROM data.jingdong_day_price_nineCounty where commodityUrl = %(url)s"
    else:
        sql_cmd = "SELECT updateTime, dayPrice FROM data.tianmao_day_price_nineCounty where commodityId = %(url)s"
    df = pd.read_sql(sql_cmd, engine, params={'url': url})
    df.drop(index=df[df['dayPrice']<=0].index,inplace=True)
    df = fillPrice(df)
    df['dayPrice'] = df['dayPrice'].apply(lambda x: (x / w))
    mov_df(df, 1)
    mov_rate(df)
    x_train = train(df)[0]
    y_train = train(df)[1]
    x_predict = train(df)[2]
    rf = RandomForestRegressor(n_jobs = -1, max_depth = 5)
    boruta = BorutaPy(
       estimator = rf, 
       n_estimators = 'auto',
       max_iter = 100 # number of trials to perform
    )
    # 模型训练
    boruta.fit(np.array(x_train), np.array(y_train))
    # 输出结果
    green_area = x_train.columns[boruta.support_].to_list()
    blue_area  = x_train.columns[boruta.support_weak_].to_list()
    # 选择最优参数
    # setup regressor
    xgb_model = xgb.XGBRegressor() 
    # performance a grid search
    tweaked_model = GridSearchCV(
        xgb_model,   
        {
            'max_depth':[1,2,5,10,20],
            'n_estimators':[20,30,50,70,100],
            'learning_rate':[0.1,0.2,0.3,0.4,0.5]
        },   
        cv = 3,   
        verbose = 1,
        n_jobs = -1,  
        scoring = 'neg_median_absolute_error')
    tweaked_model.fit(x_train, y_train)
    model1 = xgb.XGBRegressor(learning_rate = tweaked_model.best_params_['learning_rate'], 
                              max_depth = tweaked_model.best_params_['max_depth'], 
                              n_estimators = tweaked_model.best_params_['n_estimators'])
    model1.fit(x_train,y_train)
    Y_predict = model1.predict(x_predict).round(2)

    pre_date = pd.period_range((datetime.datetime.now() + relativedelta(months=1)), periods=4, freq='M')
    predict = pd.DataFrame(list(zip(pre_date,Y_predict)), columns = ['predict_date','predict_price'])
    predict['product'] = product
    predict['date'] = datetime.datetime.now().strftime('%Y-%m')
    order = ['product','date','predict_date', 'predict_price']
    predict = predict[order]
    predict['predict_price'] = predict['predict_price'].round(2)
    predict['predict_date'] = predict['predict_date'].apply(lambda x: str(x))
    predict['predict_date'] = pd.to_datetime(predict['predict_date'])
    predict['predict_date'] = predict['predict_date'].apply(lambda x: x.strftime('%Y-%m'))
    return predict

In [8]:
aa = predict_online('https://item.jd.com/10033573567269.html', '巢湖稻虾米', 10)

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    1.3s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [6]:
result = engine.execute('TRUNCATE TABLE data.price_predict_long')
data = pd.read_csv('allProduct.csv', encoding='utf-8')
for row in data.iterrows():
    try:
        table = predict_online(row[1]['url'], row[1]['product'], row[1]['weight'])
        table.to_sql('price_predict_long', con= engine, if_exists= 'append', index=False)
    except:
        print(row[1]['product'])

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    4.1s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    1.3s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    1.4s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    1.4s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 344 out of 375 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    1.4s finished
D:\Anaconda\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
